# XAI(Explainable AI): Kernel SHAP for Classification

본 핸즈온에서는 앙상블과 같은 블랙 박스 모델을 설명하는 데 적합한 SHAP(SHapley Additive exPlanations)을 사용하는 예시를 보여줍니다. SHAP은 전체 셋의 feature importance가 아니라 각 샘플 데이터마다 예측에 얼마나 기여했는지 정량화가 가능합니다.

## SHAP(SHapley Additive exPlanations)

SHAP에 대한 심화 주제는 아래 논문과 링크를 참조하세요
- [A Unified Approach to Interpreting Model Predictions] Lundberg, Scott M., and Su-In Lee Advances in Neural Information Processing Systems. 2017.
- Interpretable ML Book (SHAP chapter): http://christophm.github.io/interpretable-ml-book/shap.html

In [ ]:
%load_ext autoreload
%autoreload 2
!pip install -qU shap

In [ ]:
from autogluon.tabular import TabularDataset, TabularPredictor
import pandas as pd
import numpy as np
import sklearn
import shap
shap.initjs()

import warnings
warnings.filterwarnings('ignore')

<br>

## 1. Data preparation and Training

In [ ]:
N_SUBSAMPLE = 500  # subsample datasets for faster demo
N_TEST = 50
NSHAP_SAMPLES = 10  # how many samples to use to approximate each Shapely value, larger values will be slower

train_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/train.csv')  # can be local CSV file as well, returns Pandas DataFrame
train_data = train_data.sample(N_SUBSAMPLE)
test_data = TabularDataset('https://autogluon.s3.amazonaws.com/datasets/Inc/test.csv')
test_data = test_data.sample(N_TEST)

label = 'class'

y_train = train_data[label]
y_test = test_data[label]
X_train = pd.DataFrame(train_data.drop(columns=[label]))
X_test = pd.DataFrame(test_data.drop(columns=[label]))

display(train_data.head())

In [ ]:
predictor = TabularPredictor(label=label, problem_type='binary').fit(train_data, time_limit=20)

<br>

## 2. Explain predictions

SHAP은 각 피쳐가 예측 결과에 "얼마나" 기여하는지 설명합니다. 구체적으로 baseline에서 positive 클래스의 예측 확률 간의 편차로 정량화되며,
신규 데이터에 대한 예측 시에는 훈련 데이터에 대한 평균 예측과 다른 각 피쳐가 예측에 얼마나 기여하는지 정량화합니다.

In [ ]:
class AutogluonWrapper:
    def __init__(self, predictor, feature_names):
        self.ag_model = predictor
        self.feature_names = feature_names
    
    def predict_proba(self, X):
        if isinstance(X, pd.Series):
            X = X.values.reshape(1,-1)
        if not isinstance(X, pd.DataFrame):
            X = pd.DataFrame(X, columns=self.feature_names)
        return self.ag_model.predict_proba(X)

피쳐의 baseline reference 값을 정의합니다. 

In [ ]:
baseline = X_train.sample(100) # X_train.mode() could also be reasonable baseline for both numerical/categorical features rather than an entire dataset.

AutoGluon 예측 결과를 설명하기 위해 Kernel SHAP 값을 반환하는 KernelExplainer를 생성합니다.

In [ ]:
ag_wrapper = AutogluonWrapper(predictor, X_train.columns)
explainer = shap.KernelExplainer(ag_wrapper.predict_proba, baseline)
print("Baseline prediction: ", np.mean(ag_wrapper.predict_proba(baseline)))  # this is the same as explainer.expected_value

### SHAP for single datapoint

훈련 데이터셋 내의 임의의 데이터 포인트에 대해 SHAP을 plot해 보겠습니다.

In [ ]:
ROW_INDEX = 0  # index of an example datapoint
single_datapoint = X_train.iloc[[ROW_INDEX]]
single_prediction = ag_wrapper.predict_proba(single_datapoint)
shap_values_single = explainer.shap_values(single_datapoint, nsamples=NSHAP_SAMPLES)

In [ ]:
shap.force_plot(explainer.expected_value[0], shap_values_single[0], X_train.iloc[ROW_INDEX,:])

In [ ]:
shap.force_plot(explainer.expected_value[1], shap_values_single[1], X_train.iloc[ROW_INDEX,:])

### SHAP for dataset

테스트 데이터셋의 모든 데이터 포인트에 대해서도 SHAP을 plot할 수 있습니다.

In [ ]:
shap_values = explainer.shap_values(X_test, nsamples=NSHAP_SAMPLES)
shap.force_plot(explainer.expected_value[0], shap_values[0], X_test)

In [ ]:
shap.summary_plot(shap_values, X_test)

In [ ]:
shap.summary_plot(shap_values[0], X_test)

In [ ]:
shap.dependence_plot("education-num", shap_values[0], X_test)

### Overall Feature Importance 

개별 예측을 설명하는 대신 각 피쳐가 AutoGluon의 일반적인 예측 정확도에 얼마나 기여하는지 알고 싶다면 Permutation Shuffling을 활용할 수 있습니다.

In [ ]:
predictor.feature_importance(test_data)

<br>

## 3. Multiclass Classification
다중(multi) 클래스 분류도 SHAP 적용이 가능합니다. 이번에는 개인 소득 대신 가족 관계(relationshop)를 예측하는 문제로 변경해서 훈련을 수행 후 SHAP을 확인해 보겠습니다.

In [ ]:
label = 'relationship'

y_train = train_data[label]
y_test = test_data[label]
X_train = pd.DataFrame(train_data.drop(columns=[label]))
X_test = pd.DataFrame(test_data.drop(columns=[label]))

display(train_data.head())
print("Possible classes: \n", train_data[label].value_counts())

`problem_type`을 지정하지 않아도 AutoGluon에서 자동으로 처리하지만, 안전하게 `problem_type=multiclass`로 지정합니다.

In [ ]:
predictor_multi = TabularPredictor(label=label, problem_type='multiclass').fit(train_data, time_limit=20)

In [ ]:
baseline = X_train.sample(100) # X_train.mode() could also be reasonable baseline for both numerical/categorical features rather than an entire dataset.

ag_wrapper = AutogluonWrapper(predictor_multi, X_train.columns)
explainer = shap.KernelExplainer(ag_wrapper.predict_proba, baseline)

In [ ]:
pd.DataFrame(np.mean(ag_wrapper.predict_proba(baseline),axis=0))

In [ ]:
print("Class Info: \n", predictor_multi.class_labels)

NSHAP_SAMPLES = 10  # how many samples to use to approximate each Shapely value, larger values will be slower
shap.initjs()

class 중 Not-in-family에 대해서 SHAP을 plot해 보겠습니다.

In [ ]:
ROW_INDEX = 0  # index of an example datapoint
class_of_interest = ' Not-in-family'  # can be any value in set(y_train)
class_index = predictor_multi.class_labels.index(class_of_interest)

single_datapoint = X_train.iloc[[ROW_INDEX]]
single_prediction = ag_wrapper.predict_proba(single_datapoint)

shap_values_single = explainer.shap_values(single_datapoint, nsamples=NSHAP_SAMPLES)
print("Shapely values: \n", {predictor_multi.class_labels[i]:shap_values_single[i] for i in range(len(predictor_multi.class_labels))})

print(f"Force_plot for class: {class_of_interest}")
shap.force_plot(explainer.expected_value[class_index], shap_values_single[class_index], X_train.iloc[ROW_INDEX,:])

In [ ]:
shap_values = explainer.shap_values(X_test, nsamples=NSHAP_SAMPLES)

print(f"Force_plot for class: {class_of_interest}")
shap.force_plot(explainer.expected_value[class_index], shap_values[class_index], X_test)

In [ ]:
shap.summary_plot(shap_values, X_test)
print({"Class "+str(i) : predictor_multi.class_labels[i] for i in range(len(predictor_multi.class_labels))})

In [ ]:
dependence_feature = "marital-status"
print(f"Dependence_plot for class: {class_of_interest}  and for feature: {dependence_feature} \n")

shap.dependence_plot(dependence_feature, shap_values[class_index], X_test)

In [ ]:
predictor.feature_importance(test_data)